# Training

In [1]:
import numpy as np
import pandas as pd

In [2]:
def one_hot_encoded(y_train, classes = 0):
    one_hot = np.zeros((len(y_train), classes))
    array = y_train
    if type(array) is not np.ndarray: 
        array = np.array(y_train)
    for i in range(len(y_train)):
        one_hot[i][array[i]] = 1.0
    return one_hot

In [3]:
class Model:
    
    # Constructor
    def __init__(self):
        self.x = []
        self.y = []
        self.num_layers = 0
        self.layers_info = {}
        self.layers = []
        self.net_sum = []
        self.weights = []
        self.biases = []
        self.jacobian_weights = []
        self.jacobian_biases = []
    
    # Add a fully connected layer
    def add_dense_layer(self, size, input_size = 0, activation = 'linear'):
        self.num_layers += 1
        self.layers_info['Layer_'+str(self.num_layers)] = {
            'size': size,
            'activation': activation
        }
        self.layers.append(np.ones(size))
        self.net_sum.append(np.ones(size))
        if input_size != 0:
            self.weights.append(np.random.randn(size, input_size) * np.sqrt(2/(size+input_size)))
        else:
            input_size = self.layers_info['Layer_'+str(self.num_layers-1)]['size']
            self.weights.append(np.random.randn(size, input_size) * np.sqrt(2/(size+input_size)))
        self.biases.append(np.random.randn(size) * np.sqrt(1/size))
        
    def mse(self, true, pred):
        return ((true-pred)**2)/2
        
    # Selection of Loss Function and Optimization function
    def set_parameters(self, lr = 0.01, loss = 'mse'):
        if loss == 'mse':
            self.loss = loss
            self.loss_function = self.mse
        self.lr = lr
    
    # Sigmoid
    def sigmoid(self, x):
        return 1 / (1 + (np.e ** -x))
    
    # ReLU - Rectified Linear Unit
    def relu(self, x):
        return max(0, x)
    
    # Softmax
    def softmax(self, x):
        exp = np.exp(x)
        return np.true_divide(exp, sum(exp)).transpose()
    
    def activation(self, net_sum, activation):
        if activation == 'sigmoid':
            activation_function = self.sigmoid
            output_vector = np.array([activation_function(i) for i in net_sum]).transpose()
        elif activation == 'relu':
            activation_function = self.relu
            output_vector = np.array([activation_function(i) for i in net_sum]).transpose()
        elif activation == 'softmax':
            activation_function = self.softmax
            output_vector = activation_function(net_sum)
        return output_vector
    
    # Forward Propogation
    def neural_network_output(self, record):
        input_vector = record.transpose()
        output_vector = None
        for i in range(len(self.layers)):
            # y = aW + b
            self.net_sum[i] = np.matmul(self.weights[i], input_vector) + self.biases[i].transpose()
            self.layers[i] = self.activation(self.net_sum[i], self.layers_info['Layer_'+str(i+1)]['activation'])
            input_vector = self.layers[i]
        output_vector = self.layers[len(self.layers)-1]
        return output_vector
    
    # For output layer
    # delta = dE/dnet_sum_output = dE/doutput * doutput/dnet_sum_output
    # dE/doutput
    def output_loss_derivative(self, true, pred):
        if self.loss == 'mse':
            return (true - pred)
    
    # doutput/dnet_sum_output
    def activation_derivative(self, output_vector_i, net_sum_i, activation):
        if activation == 'relu':
            return np.array([1 if i>0 else 0 for i in net_sum_i])
        elif activation == 'sigmoid':
            return output_vector_i * (1 - output_vector_i)
        elif activation == 'softmax':
            return output_vector_i * (1 - output_vector_i)
        else:
            return output_vector_i * (1 - output_vector_i)
    
    # For hidden layer
    # dE/dnet_sum_hidden = dotproduct(weights_ji, delta_o)
    def hidden_loss_derivative(self, grad_next, layer_number, num_of_neurons_next):
        grad = []
        for i in range(num_of_neurons_next):
            weights_next = self.weights[layer_number+1][:, i]
            grad.append(np.dot(weights_next, grad_next))
        return np.array(grad)
    
    # Training using Backpropogation
    def train(self, X_train, y_train, epochs = 1, batch_size = 0): 
        
        # initializing Jacobian
        for i in range(self.num_layers):
            size = self.layers_info['Layer_'+str(i+1)]['size']            
            input_size = 0
            self.jacobian_biases.append(np.zeros((size)))
            if i == 0:
                input_size = X_train.shape[1]
            else:
                input_size = len(self.layers[i-1])
            self.jacobian_weights.append(np.zeros((size, input_size)))
        
        # Creation of batches
        self.x = X_train
        self.y = y_train
        if batch_size == 0:
            batch_size = len(y_train)
        batches = int(abs(len(X_train) / batch_size))
        X_train = np.array_split(X_train, batches)
        y_train = np.array_split(y_train, batches)
        
        for epoch in range(epochs): 
            for batch in range(batches):
                for record, label in zip(X_train[batch], y_train[batch]):
                    true_output = np.array([label])
                    predicted_output = self.neural_network_output(np.array(record))

                    # gradient of output layer
                    output_vector_i = self.layers[self.num_layers-1]
                    net_sum_i = self.net_sum[self.num_layers-1]
                    activation = self.layers_info['Layer_'+str(self.num_layers)]['activation']
                    # derivative of output loss * derivetive of activation
                    grad_o = self.output_loss_derivative(true_output, predicted_output) * self.activation_derivative(output_vector_i, net_sum_i, activation)
                    
                    # Jacobian Output Layer - Jw = input * grad_o and Jb = 1.0 * grad_o
                    if self.num_layers-2 < 0:
                        self.jacobian_weights[self.num_layers-1] += np.array(record) * grad_o
                    
                    self.jacobian_weights[self.num_layers-1] += grad_o[:, None] @ self.layers[self.num_layers-2][None, :]
                    self.jacobian_biases[self.num_layers-1] += grad_o
                    
                    # gradient of hidden layer
                    grad_next = grad_o
                    for i in range(self.num_layers-2, -1, -1):
                        output_vector_i = self.layers[i]
                        net_sum_i = self.layers[i]
                        activation = self.layers_info['Layer_'+str(i+1)]['activation']
                        num_of_neurons_next = self.layers_info['Layer_'+str(i+2)]['size']
                        
                        # derivative of hidden loss * derivetive of activation
                        grad_h = self.hidden_loss_derivative(grad_next, i, num_of_neurons_next) * self.activation_derivative(output_vector_i, net_sum_i, activation)
                        
                        # Jacobian Hidden layer - Jw = input * grad_next and Jb = 1.0 * grad_next
                        if i-1 < 0:
                            self.jacobian_weights[i] += grad_h[:, None] @ np.array(record)[None, :]
                        self.jacobian_weights[i] += grad_h[:, None] @ self.layers[i-1][None, :]
                        self.jacobian_biases[i] += grad_h
                        
                        # change the gradient
                        grad_next = grad_h
                
                # Divide accumulated jacobian by number of records in the batch
                for i in range(self.num_layers):
                    self.jacobian_weights[i] = self.jacobian_weights[i] / len(record)
                    self.jacobian_biases[i] = self.jacobian_biases[i] / len(record)
                
                # Update weights and biases
                for i in range(self.num_layers-1, -1, -1):
                    self.weights[i] += self.lr * self.jacobian_weights[i]
                    self.biases[i] += self.lr * self.jacobian_biases[i]
            
            # Accuracy and Loss
            print('Epoch ',epoch)
            self.evaluate(self.x, self.y)
            print('')
            
    
    # Predict
    def predict(self, X_test):
        predictions = []
        classes = self.layers_info['Layer_'+str(self.num_layers)]['size']
        for record in X_test:
            predicted_output = self.neural_network_output(record)
            if classes == 1:
                if predicted_output[0] >= 0.5:
                    predictions.append(1)
                else:
                    predictions.append(0)
            else:
                predict = np.argmax(predicted_output)
                predictions.append(predict)
        return predictions
    
    # Accuracy and Loss
    def evaluate(self, X_test, y_test):       
        # Loss
        loss = 0
        for record, label in zip(X_test, y_test):
            true_output = label
            predicted_output = self.neural_network_output(record)
            loss += self.loss_function(true_output, predicted_output)
        print('Training Loss: ',loss/len(X_test))
        
        # Accuracy
        true_output = y_test
        predicted_output = self.predict(X_test)
        x = np.logical_not(np.logical_xor(true_output, predicted_output))
        accuracy = len(x[x==True])/len(x)
        print('Training Accuracy: ',accuracy)
            
    
    

# Testing

Feed-froward Neural Network

In [21]:
model = Model()

model.add_dense_layer(4, 3, activation='relu')
model.add_dense_layer(1, activation='sigmoid')

Creating Dataset - XOR Dataset

In [5]:
X_train = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
y_train = np.array([0, 1, 1, 1, 1, 0, 0])

Setting Hyperparameters: <br>
Loss Function: Mean Square Error(mse) <br> 
Learning rate: 0.01

In [22]:
model.set_parameters(lr=0.1, loss='mse')

Gradient Descent Optimization

In [7]:
model.train(X_train, y_train, epochs=1000)

Epoch  0
Training Loss:  [0.13019199]
Training Accuracy:  0.2857142857142857

Epoch  1
Training Loss:  [0.12986797]
Training Accuracy:  0.2857142857142857

Epoch  2
Training Loss:  [0.12953848]
Training Accuracy:  0.2857142857142857

Epoch  3
Training Loss:  [0.12922367]
Training Accuracy:  0.42857142857142855

Epoch  4
Training Loss:  [0.12892888]
Training Accuracy:  0.42857142857142855

Epoch  5
Training Loss:  [0.12865461]
Training Accuracy:  0.42857142857142855

Epoch  6
Training Loss:  [0.12839982]
Training Accuracy:  0.42857142857142855

Epoch  7
Training Loss:  [0.12816306]
Training Accuracy:  0.42857142857142855

Epoch  8
Training Loss:  [0.12794284]
Training Accuracy:  0.42857142857142855

Epoch  9
Training Loss:  [0.12773776]
Training Accuracy:  0.42857142857142855

Epoch  10
Training Loss:  [0.12754651]
Training Accuracy:  0.42857142857142855

Epoch  11
Training Loss:  [0.12736791]
Training Accuracy:  0.42857142857142855

Epoch  12
Training Loss:  [0.12720088]
Training Accur

Epoch  116
Training Loss:  [0.12912188]
Training Accuracy:  0.5714285714285714

Epoch  117
Training Loss:  [0.12916886]
Training Accuracy:  0.5714285714285714

Epoch  118
Training Loss:  [0.12912486]
Training Accuracy:  0.5714285714285714

Epoch  119
Training Loss:  [0.12906101]
Training Accuracy:  0.5714285714285714

Epoch  120
Training Loss:  [0.12898918]
Training Accuracy:  0.5714285714285714

Epoch  121
Training Loss:  [0.12891329]
Training Accuracy:  0.5714285714285714

Epoch  122
Training Loss:  [0.12883462]
Training Accuracy:  0.5714285714285714

Epoch  123
Training Loss:  [0.12875357]
Training Accuracy:  0.5714285714285714

Epoch  124
Training Loss:  [0.12867022]
Training Accuracy:  0.5714285714285714

Epoch  125
Training Loss:  [0.12858457]
Training Accuracy:  0.5714285714285714

Epoch  126
Training Loss:  [0.12849659]
Training Accuracy:  0.5714285714285714

Epoch  127
Training Loss:  [0.12840626]
Training Accuracy:  0.5714285714285714

Epoch  128
Training Loss:  [0.12831351]


Epoch  227
Training Loss:  [0.10762781]
Training Accuracy:  0.5714285714285714

Epoch  228
Training Loss:  [0.1074405]
Training Accuracy:  0.5714285714285714

Epoch  229
Training Loss:  [0.10727527]
Training Accuracy:  0.5714285714285714

Epoch  230
Training Loss:  [0.10701203]
Training Accuracy:  0.5714285714285714

Epoch  231
Training Loss:  [0.10687841]
Training Accuracy:  0.5714285714285714

Epoch  232
Training Loss:  [0.10660558]
Training Accuracy:  0.5714285714285714

Epoch  233
Training Loss:  [0.10648806]
Training Accuracy:  0.5714285714285714

Epoch  234
Training Loss:  [0.10624066]
Training Accuracy:  0.5714285714285714

Epoch  235
Training Loss:  [0.10602385]
Training Accuracy:  0.5714285714285714

Epoch  236
Training Loss:  [0.1057774]
Training Accuracy:  0.5714285714285714

Epoch  237
Training Loss:  [0.10558848]
Training Accuracy:  0.5714285714285714

Epoch  238
Training Loss:  [0.10543129]
Training Accuracy:  0.5714285714285714

Epoch  239
Training Loss:  [0.10516618]
Tr

Epoch  343
Training Loss:  [0.08861154]
Training Accuracy:  0.8571428571428571

Epoch  344
Training Loss:  [0.08837433]
Training Accuracy:  0.8571428571428571

Epoch  345
Training Loss:  [0.08831076]
Training Accuracy:  0.8571428571428571

Epoch  346
Training Loss:  [0.08815143]
Training Accuracy:  0.8571428571428571

Epoch  347
Training Loss:  [0.08797566]
Training Accuracy:  0.8571428571428571

Epoch  348
Training Loss:  [0.0878683]
Training Accuracy:  0.8571428571428571

Epoch  349
Training Loss:  [0.08769887]
Training Accuracy:  0.8571428571428571

Epoch  350
Training Loss:  [0.0877074]
Training Accuracy:  0.8571428571428571

Epoch  351
Training Loss:  [0.08746102]
Training Accuracy:  0.8571428571428571

Epoch  352
Training Loss:  [0.08730764]
Training Accuracy:  0.8571428571428571

Epoch  353
Training Loss:  [0.08725613]
Training Accuracy:  0.8571428571428571

Epoch  354
Training Loss:  [0.08703007]
Training Accuracy:  0.8571428571428571

Epoch  355
Training Loss:  [0.0869432]
Tra

Epoch  452
Training Loss:  [0.07271656]
Training Accuracy:  0.8571428571428571

Epoch  453
Training Loss:  [0.0725178]
Training Accuracy:  0.8571428571428571

Epoch  454
Training Loss:  [0.07247583]
Training Accuracy:  0.8571428571428571

Epoch  455
Training Loss:  [0.07227473]
Training Accuracy:  0.8571428571428571

Epoch  456
Training Loss:  [0.07215869]
Training Accuracy:  0.8571428571428571

Epoch  457
Training Loss:  [0.07194269]
Training Accuracy:  0.8571428571428571

Epoch  458
Training Loss:  [0.07172403]
Training Accuracy:  0.8571428571428571

Epoch  459
Training Loss:  [0.07180805]
Training Accuracy:  0.8571428571428571

Epoch  460
Training Loss:  [0.0714399]
Training Accuracy:  0.8571428571428571

Epoch  461
Training Loss:  [0.07128167]
Training Accuracy:  0.8571428571428571

Epoch  462
Training Loss:  [0.07118924]
Training Accuracy:  0.8571428571428571

Epoch  463
Training Loss:  [0.07100441]
Training Accuracy:  0.8571428571428571

Epoch  464
Training Loss:  [0.0708567]
Tra

Epoch  562
Training Loss:  [0.04945919]
Training Accuracy:  1.0

Epoch  563
Training Loss:  [0.04911393]
Training Accuracy:  1.0

Epoch  564
Training Loss:  [0.04905459]
Training Accuracy:  1.0

Epoch  565
Training Loss:  [0.04864053]
Training Accuracy:  1.0

Epoch  566
Training Loss:  [0.04829924]
Training Accuracy:  1.0

Epoch  567
Training Loss:  [0.04797423]
Training Accuracy:  1.0

Epoch  568
Training Loss:  [0.04790529]
Training Accuracy:  1.0

Epoch  569
Training Loss:  [0.04750837]
Training Accuracy:  1.0

Epoch  570
Training Loss:  [0.04716952]
Training Accuracy:  1.0

Epoch  571
Training Loss:  [0.04684616]
Training Accuracy:  1.0

Epoch  572
Training Loss:  [0.04674453]
Training Accuracy:  1.0

Epoch  573
Training Loss:  [0.04637601]
Training Accuracy:  1.0

Epoch  574
Training Loss:  [0.04604313]
Training Accuracy:  1.0

Epoch  575
Training Loss:  [0.04572142]
Training Accuracy:  1.0

Epoch  576
Training Loss:  [0.04558527]
Training Accuracy:  1.0

Epoch  577
Training Loss:

Epoch  767
Training Loss:  [0.01309427]
Training Accuracy:  1.0

Epoch  768
Training Loss:  [0.0130026]
Training Accuracy:  1.0

Epoch  769
Training Loss:  [0.01292942]
Training Accuracy:  1.0

Epoch  770
Training Loss:  [0.01288789]
Training Accuracy:  1.0

Epoch  771
Training Loss:  [0.01279904]
Training Accuracy:  1.0

Epoch  772
Training Loss:  [0.01272621]
Training Accuracy:  1.0

Epoch  773
Training Loss:  [0.01268833]
Training Accuracy:  1.0

Epoch  774
Training Loss:  [0.01260057]
Training Accuracy:  1.0

Epoch  775
Training Loss:  [0.01252739]
Training Accuracy:  1.0

Epoch  776
Training Loss:  [0.01249501]
Training Accuracy:  1.0

Epoch  777
Training Loss:  [0.01240701]
Training Accuracy:  1.0

Epoch  778
Training Loss:  [0.01233288]
Training Accuracy:  1.0

Epoch  779
Training Loss:  [0.01230746]
Training Accuracy:  1.0

Epoch  780
Training Loss:  [0.01221818]
Training Accuracy:  1.0

Epoch  781
Training Loss:  [0.0121426]
Training Accuracy:  1.0

Epoch  782
Training Loss:  

Epoch  994
Training Loss:  [0.00543283]
Training Accuracy:  1.0

Epoch  995
Training Loss:  [0.00540741]
Training Accuracy:  1.0

Epoch  996
Training Loss:  [0.00539416]
Training Accuracy:  1.0

Epoch  997
Training Loss:  [0.00537611]
Training Accuracy:  1.0

Epoch  998
Training Loss:  [0.00537]
Training Accuracy:  1.0

Epoch  999
Training Loss:  [0.00534599]
Training Accuracy:  1.0



Model Evaluation

In [8]:
print(model.predict(X_train), model.predict(np.array([[1,1,0]])))
model.evaluate(X_train, y_train)

[0, 1, 1, 1, 1, 0, 0] [0]
Training Loss:  [0.00534599]
Training Accuracy:  1.0


Stochastic Gradient Desccent Optimization 

In [23]:
model.train(X_train, y_train, epochs=1000, batch_size=1)

Epoch  0
Training Loss:  [0.14291094]
Training Accuracy:  0.2857142857142857

Epoch  1
Training Loss:  [0.14252709]
Training Accuracy:  0.2857142857142857

Epoch  2
Training Loss:  [0.14218063]
Training Accuracy:  0.2857142857142857

Epoch  3
Training Loss:  [0.14187051]
Training Accuracy:  0.2857142857142857

Epoch  4
Training Loss:  [0.14159583]
Training Accuracy:  0.2857142857142857

Epoch  5
Training Loss:  [0.14135586]
Training Accuracy:  0.2857142857142857

Epoch  6
Training Loss:  [0.14114996]
Training Accuracy:  0.2857142857142857

Epoch  7
Training Loss:  [0.14097761]
Training Accuracy:  0.2857142857142857

Epoch  8
Training Loss:  [0.14083839]
Training Accuracy:  0.2857142857142857

Epoch  9
Training Loss:  [0.14073194]
Training Accuracy:  0.2857142857142857

Epoch  10
Training Loss:  [0.14065794]
Training Accuracy:  0.2857142857142857

Epoch  11
Training Loss:  [0.14061611]
Training Accuracy:  0.42857142857142855

Epoch  12
Training Loss:  [0.14060618]
Training Accuracy:  0.

Epoch  205
Training Loss:  [0.08091968]
Training Accuracy:  0.8571428571428571

Epoch  206
Training Loss:  [0.08071786]
Training Accuracy:  0.8571428571428571

Epoch  207
Training Loss:  [0.08043216]
Training Accuracy:  0.8571428571428571

Epoch  208
Training Loss:  [0.08000619]
Training Accuracy:  0.8571428571428571

Epoch  209
Training Loss:  [0.07972961]
Training Accuracy:  0.8571428571428571

Epoch  210
Training Loss:  [0.07951779]
Training Accuracy:  0.8571428571428571

Epoch  211
Training Loss:  [0.07902569]
Training Accuracy:  0.8571428571428571

Epoch  212
Training Loss:  [0.0789056]
Training Accuracy:  0.8571428571428571

Epoch  213
Training Loss:  [0.07840537]
Training Accuracy:  0.8571428571428571

Epoch  214
Training Loss:  [0.07827942]
Training Accuracy:  0.8571428571428571

Epoch  215
Training Loss:  [0.07778819]
Training Accuracy:  0.8571428571428571

Epoch  216
Training Loss:  [0.07765328]
Training Accuracy:  0.8571428571428571

Epoch  217
Training Loss:  [0.07722178]
T

Epoch  309
Training Loss:  [0.04229116]
Training Accuracy:  0.8571428571428571

Epoch  310
Training Loss:  [0.04188564]
Training Accuracy:  0.8571428571428571

Epoch  311
Training Loss:  [0.04148455]
Training Accuracy:  0.8571428571428571

Epoch  312
Training Loss:  [0.04121197]
Training Accuracy:  0.8571428571428571

Epoch  313
Training Loss:  [0.04082855]
Training Accuracy:  0.8571428571428571

Epoch  314
Training Loss:  [0.04041497]
Training Accuracy:  1.0

Epoch  315
Training Loss:  [0.04011331]
Training Accuracy:  1.0

Epoch  316
Training Loss:  [0.03973004]
Training Accuracy:  1.0

Epoch  317
Training Loss:  [0.03934022]
Training Accuracy:  1.0

Epoch  318
Training Loss:  [0.03897607]
Training Accuracy:  1.0

Epoch  319
Training Loss:  [0.03871939]
Training Accuracy:  1.0

Epoch  320
Training Loss:  [0.03834958]
Training Accuracy:  1.0

Epoch  321
Training Loss:  [0.03798745]
Training Accuracy:  1.0

Epoch  322
Training Loss:  [0.03770087]
Training Accuracy:  1.0

Epoch  323
Trai

Epoch  519
Training Loss:  [0.01223756]
Training Accuracy:  1.0

Epoch  520
Training Loss:  [0.01217519]
Training Accuracy:  1.0

Epoch  521
Training Loss:  [0.01214816]
Training Accuracy:  1.0

Epoch  522
Training Loss:  [0.01210053]
Training Accuracy:  1.0

Epoch  523
Training Loss:  [0.01206969]
Training Accuracy:  1.0

Epoch  524
Training Loss:  [0.01199349]
Training Accuracy:  1.0

Epoch  525
Training Loss:  [0.01197351]
Training Accuracy:  1.0

Epoch  526
Training Loss:  [0.01195506]
Training Accuracy:  1.0

Epoch  527
Training Loss:  [0.01191866]
Training Accuracy:  1.0

Epoch  528
Training Loss:  [0.01182165]
Training Accuracy:  1.0

Epoch  529
Training Loss:  [0.01181429]
Training Accuracy:  1.0

Epoch  530
Training Loss:  [0.01176735]
Training Accuracy:  1.0

Epoch  531
Training Loss:  [0.01173701]
Training Accuracy:  1.0

Epoch  532
Training Loss:  [0.01163996]
Training Accuracy:  1.0

Epoch  533
Training Loss:  [0.01162219]
Training Accuracy:  1.0

Epoch  534
Training Loss:

Epoch  710
Training Loss:  [0.00709979]
Training Accuracy:  1.0

Epoch  711
Training Loss:  [0.00708549]
Training Accuracy:  1.0

Epoch  712
Training Loss:  [0.00708481]
Training Accuracy:  1.0

Epoch  713
Training Loss:  [0.00705829]
Training Accuracy:  1.0

Epoch  714
Training Loss:  [0.00704816]
Training Accuracy:  1.0

Epoch  715
Training Loss:  [0.00702585]
Training Accuracy:  1.0

Epoch  716
Training Loss:  [0.00702262]
Training Accuracy:  1.0

Epoch  717
Training Loss:  [0.00699835]
Training Accuracy:  1.0

Epoch  718
Training Loss:  [0.0069881]
Training Accuracy:  1.0

Epoch  719
Training Loss:  [0.00696551]
Training Accuracy:  1.0

Epoch  720
Training Loss:  [0.00695768]
Training Accuracy:  1.0

Epoch  721
Training Loss:  [0.00694197]
Training Accuracy:  1.0

Epoch  722
Training Loss:  [0.00693436]
Training Accuracy:  1.0

Epoch  723
Training Loss:  [0.0069169]
Training Accuracy:  1.0

Epoch  724
Training Loss:  [0.00689988]
Training Accuracy:  1.0

Epoch  725
Training Loss:  

Epoch  914
Training Loss:  [0.00494517]
Training Accuracy:  1.0

Epoch  915
Training Loss:  [0.00493767]
Training Accuracy:  1.0

Epoch  916
Training Loss:  [0.00493461]
Training Accuracy:  1.0

Epoch  917
Training Loss:  [0.00493012]
Training Accuracy:  1.0

Epoch  918
Training Loss:  [0.00492333]
Training Accuracy:  1.0

Epoch  919
Training Loss:  [0.00490812]
Training Accuracy:  1.0

Epoch  920
Training Loss:  [0.00490293]
Training Accuracy:  1.0

Epoch  921
Training Loss:  [0.00489735]
Training Accuracy:  1.0

Epoch  922
Training Loss:  [0.00489498]
Training Accuracy:  1.0

Epoch  923
Training Loss:  [0.00488853]
Training Accuracy:  1.0

Epoch  924
Training Loss:  [0.00487244]
Training Accuracy:  1.0

Epoch  925
Training Loss:  [0.00486941]
Training Accuracy:  1.0

Epoch  926
Training Loss:  [0.00486305]
Training Accuracy:  1.0

Epoch  927
Training Loss:  [0.00486137]
Training Accuracy:  1.0

Epoch  928
Training Loss:  [0.00485529]
Training Accuracy:  1.0

Epoch  929
Training Loss:

Model Evaluation

In [24]:
print(model.predict(X_train), model.predict(np.array([[1,1,0]])))
model.evaluate(X_train, y_train)

[0, 1, 1, 1, 1, 0, 0] [0]
Training Loss:  [0.00441694]
Training Accuracy:  1.0
